In [1]:
from ARRG import *

In [2]:
class ObservableDataset(Dataset):
	"""
	Converts observable dataset into PyTorch syntax.
	"""
	def __init__(self, data):
		self.data = data

	def __len__(self):
		return self.data.shape[0]

	def __getitem__(self, idx):
		sample = self.data[idx]
		return sample

Here we do some data proccessing to extract the event-level observables we want to train with. Remember that the event level data-structure contains arrays of hadron-level kinematic data $p_x, p_y, p_z, E, m_h$. To need to compute event-level, experiment-level, or macroscopic-level observables that can actually be observed from experiment.

I'll only use hadron multiplicity but we could also generate more distributions (sphericity, thrust, shape parameters, energy correlators, etc.) if desired. Hadron multiplicity is extremely sensitive to $a$ and $b$ so we can tune these parameters essentially exclusively on hadron multiplicity.

In [3]:
# Paths to the datasets on perlmutter
exp_hadrons_PATH = '/global/cfs/projectdirs/m3246/hadronization-tune-mlrw-data/pgun_qqbar_hadrons_a_0.68_b_0.98_sigma_0.335_N_1e6.npy'
#exp_accept_reject_PATH = '/global/cfs/projectdirs/m3246/hadronization-tune-mlrw-data/pgun_qqbar_accept_reject_z_a_0.68_b_0.98_sigma_0.335_N_1e6.npy'
sim_hadrons_PATH = '/global/cfs/projectdirs/m3246/hadronization-tune-mlrw-data/pgun_qqbar_hadrons_a_0.72_b_0.88_sigma_0.335_N_1e6.npy'
sim_accept_reject_PATH = '/global/cfs/projectdirs/m3246/hadronization-tune-mlrw-data/pgun_qqbar_accept_reject_z_a_0.72_b_0.88_sigma_0.335_N_1e6.npy'

# Load the arrays
exp_hadrons       = np.load(exp_hadrons_PATH, mmap_mode="r")
sim_hadrons       = np.load(sim_hadrons_PATH, mmap_mode="r")
sim_accept_reject = np.load(sim_accept_reject_PATH, mmap_mode = "r")

# Print dataset shapes
print('Experimental observable shape:', exp_hadrons.shape)
print('Simulated observable shape:', sim_hadrons.shape)
print('Simulated z shape:', sim_accept_reject.shape)

# Restrict to a subset of the full dataset (for memory)
N_events = int(100000)

# Extract the hadron multiplicity
exp_mult = np.array([len(exp_hadrons[i,:][np.abs(exp_hadrons[i,:,0]) > 0.0]) for i in range(exp_hadrons.shape[0])])
sim_mult = np.array([len(sim_hadrons[i,:][np.abs(sim_hadrons[i,:,0]) > 0.0]) for i in range(sim_hadrons.shape[0])])
# Extract the transverse mass
sim_mT   = np.sqrt(sim_hadrons[:,:,0]**2 + sim_hadrons[:,:,1]**2 + sim_hadrons[:,:,4]**2)

# Convert into torch objects
sim_mult          = torch.Tensor(sim_mult[0:N_events].copy())
sim_accept_reject = torch.Tensor(sim_accept_reject[0:N_events].copy())
sim_mT            = torch.Tensor(sim_mT[0:N_events].copy())
exp_mult          = torch.Tensor(exp_mult[0:N_events].copy())

# Check the accepted z-values, if z == 1 reduce it by epsilon (a very nasty bug to find).
# The a-coefficient when computing the likelihood has a term propotional to log(1-z). If 
# z = 1, this term diverges to -inf and completely destroys the backward pass.
epsilon = 1e-5
sim_accept_reject[sim_accept_reject == 1] = 1 - epsilon

# Print dataset shapes
print('Experimental multiplicity shape:', exp_mult.shape)
print('Simulated multiplicity shape:', sim_mult.shape)
print('Simulated z shape:', sim_accept_reject.shape)
print('Simulated mT shape:', sim_mT.shape)

# Prepare data for DataLoader
sim_mult          = ObservableDataset(sim_mult)
sim_accept_reject = ObservableDataset(sim_accept_reject)
sim_mT            = ObservableDataset(sim_mT)
exp_mult          = ObservableDataset(exp_mult)

Experimental observable shape: (1000000, 50, 5)
Simulated observable shape: (1000000, 50, 5)
Simulated z shape: (1000000, 50, 100)
Experimental multiplicity shape: torch.Size([100000])
Simulated multiplicity shape: torch.Size([100000])
Simulated z shape: torch.Size([100000, 50, 100])
Simulated mT shape: torch.Size([100000, 50])


In [4]:
# Set batch size -- TBD: Implement batch size scheduler
batch_size = 5000

# Initialize data-loaders
sim_observable_dataloader    = DataLoader(sim_mult, batch_size = batch_size, shuffle = False)
sim_accept_reject_dataloader = DataLoader(sim_accept_reject, batch_size = batch_size, shuffle = False)
sim_mT_dataloader            = DataLoader(sim_mT, batch_size = batch_size, shuffle = False)
exp_observable_dataloader    = DataLoader(exp_mult, batch_size = batch_size, shuffle = False)

In [5]:
# Training hyperparameters
epochs = 2
over_sample_factor = 10.0
learning_rate = 0.01
# Length of event buffer
dim_multiplicity = sim_accept_reject_dataloader.dataset.data.shape[1]
dim_accept_reject = sim_accept_reject_dataloader.dataset.data.shape[2]

print('Each event has been zero-padded to a length of', dim_multiplicity)
print('Each emission has been zero-padded to a length of', dim_accept_reject)

# Define base parameters of simulated data (a, b)
params_base = torch.tensor([0.72, 0.88])
# If params_init is set equal to None, the tuned parameters are initialized to the base parameters
params_init = None
#params_init = torch.tensor([0.6, 1.5])

print_details = True
results_dir = r'./ARRG_a_b_tune_perlmutter'

Each event has been zero-padded to a length of 50
Each emission has been zero-padded to a length of 100


In [6]:
# Ensure defualt printing options
torch.set_printoptions(profile="default") # reset
# Create a training instance
ARRG = ARRG(epochs = epochs, dim_multiplicity = dim_multiplicity, dim_accept_reject = dim_accept_reject, over_sample_factor = over_sample_factor,
						   params_base = params_base, sim_observable_dataloader = sim_observable_dataloader, sim_kinematics_z_dataloader = sim_accept_reject_dataloader, 
						   sim_kinematics_mT_dataloader = sim_mT_dataloader, exp_observable_dataloader = exp_observable_dataloader, print_details = print_details, 
						   results_dir = results_dir, params_init = params_init)

tensor([0.7200, 0.8800])


  0%|                                                                         | 0/2 [00:00<?, ?it/s]

Batch # 0
----------------------------------------------
Loss: 0.001539732
Gradient of a: 0.019106671
Gradient of b: -0.010020412
Loss: 0.001540, LR: 0.010000
a: 0.7100000381469727, b: 0.8899999856948853
----------------------------------------------
Batch # 1
----------------------------------------------
Loss: 0.002046078
Gradient of a: 0.02132484
Gradient of b: -0.011841806
Loss: 0.002046, LR: 0.010000
a: 0.6999865174293518, b: 0.9000087380409241
----------------------------------------------
Batch # 2
----------------------------------------------
Loss: 0.0009167767
Gradient of a: 0.013805228
Gradient of b: -0.008636051
Loss: 0.000917, LR: 0.010000
a: 0.6902390718460083, b: 0.9098764061927795
----------------------------------------------
Batch # 3
----------------------------------------------
Loss: 0.00086725253
Gradient of a: 0.013185384
Gradient of b: -0.0068583894
Loss: 0.000867, LR: 0.010000
a: 0.6806364059448242, b: 0.9195113182067871
----------------------------------------

 50%|███████████████████████████████▌                               | 1/2 [42:41<42:41, 2561.58s/it]

Batch # 0
----------------------------------------------
Loss: 0.00020098765
Gradient of a: -0.0043144403
Gradient of b: 0.0015344885
Loss: 0.000201, LR: 0.000400
a: 0.60731041431427, b: 0.9923425912857056
----------------------------------------------
Batch # 1
----------------------------------------------
Loss: 0.00033154
Gradient of a: 0.0013025022
Gradient of b: 0.00033376197
Loss: 0.000332, LR: 0.000080
a: 0.6073084473609924, b: 0.9923626780509949
----------------------------------------------
Batch # 2
----------------------------------------------
Loss: 0.00045103848
Gradient of a: -0.008300493
Gradient of b: 0.0038167166
Loss: 0.000451, LR: 0.000080
a: 0.6073164343833923, b: 0.9923590421676636
----------------------------------------------
Batch # 3
----------------------------------------------
Loss: 0.00019651694
Gradient of a: -0.0029143838
Gradient of b: 0.0014634749
Loss: 0.000197, LR: 0.000080
a: 0.6073266267776489, b: 0.9923529624938965
---------------------------------

In [ ]:
# Set the optimizer
optimizer = torch.optim.Adam(ARRG.weight_nexus.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(macroscopic_trainer.weight_nexus.parameters(), lr=learning_rate)

# Set learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= 'min', factor = 0.2, patience = 3)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.1)
#scheduler = None

# Train!
a_b_final, a_b_search = ARRG.train_ARRG(optimizer = optimizer, scheduler = scheduler)

print("Training complete!")
print("Final parameters: a =", a_b_final[0], 'b =', a_b_final[1])